#### EXAMPLE WRAPPING POLARS

In [1]:
import polars
from coppertop.pipe import *
# from coppertop.utils import *

from bones.lang.types import btup
from dm.core.types import date, N, num
import dm.pp
from broot import *
from broot import DD

In [2]:
import builtins
from bones.ts.metatypes import weaken, BTAtom
from dm.core.types import txt, pylist, pytuple, pydict, T1
from dm.core.datetime import parseDate, toCTimeFormat


# pframe = BTAtom.ensure('pframe')
# pcol = BTAtom.ensure('pcol')

# weaken(polars.DataFrame, pframe)
# weaken(polars.Series, pcol)

pframe = polars.DataFrame
pcol = polars.Series


@coppertop(style=binary)
def take(a:pframe, k:txt) -> pframe:
    return a[[k]]

@coppertop(style=binary)
def take(a:pframe, isOrKs:pylist+pytuple) -> pframe:
    if not isOrKs:
        raise NotYetImplemented()
    elif isinstance(isOrKs[0], (builtins.str, txt)):
        return a[isOrKs]
    elif isinstance(isOrKs[0], int):
        raise NotYetImplemented()
    else:
        raise TypeError()

@coppertop(style=binary)
def to(xs:pcol, t:(N**num)[btup]) -> (N**num)[btup]:
    xs = xs.to_numpy()
    return btup(t, [parseNum(x) for x in xs])

@coppertop(style=binary)
def to(xs:pcol, t:(N**date)[btup], f:txt) -> (N**date)[btup]:
    xs = xs.to_numpy()
    cFormat = toCTimeFormat(f)
    return btup(t, [parseDate(x, cFormat) for x in xs])

@coppertop(style=ternary)
def atPut(pf:pframe, k:txt, col:pcol) -> pframe:
    # pf.replace is destructive :(, pf.with_column is non-destructive :)
    return pf.with_column(polars.Series(k, col))

@coppertop(style=ternary)
def atPut(pf:pframe, k:txt, col:(N**T1)[btup]) -> pframe:
    # pf.replace is destructive :(, pf.with_column is non-destructive :)
    return pf.with_column(polars.Series(k, col))

@coppertop
def csvTo(filename:txt, t:pframe) -> pframe:
    return polars.read_csv(filename)

@coppertop
def csvTo(filename:txt, t:pframe, renames:pydict) -> pframe:
    # rename is non-destuctive :)
    return polars.read_csv(filename).rename(renames)

@coppertop
def csvTo(filename:txt, t:pframe, renames:pydict, conversions:pydict) -> pframe:
    return polars.read_csv(filename)

@coppertop(style=unary)
def keys(d:pframe) -> pylist:
    return d.columns

In [3]:
renames = {
    'Date':'date', '1 Mo':'1m', '2 Mo':'2m', '3 Mo':'3m', '6 Mo':'6m', '1 Yr':'1y', '2 Yr':'2y', 
    '3 Yr':'3y', '5 Yr':'5y', '7 Yr':'7y', '10 Yr':'10y', '20 Yr':'20y', '30 Yr':'30y',
}

pdf = './analysis/data/US yields.csv' >> csvTo(_, pframe, renames)

In [4]:
pdf >> keys

In [5]:
pdf >> DD;

In [6]:
pdf['date'] >> DD;

In [7]:
pdf >> take >> 'date'

In [8]:
pdf2 = pdf >> atPut >> 'date' >> (pdf['date'] >> to(_, _, 'MM/DD/YY') >> (N**date)[btup])

In [9]:
pdf2 >> take >> ['date', '1y', '5y']

In [10]:
pdf >> take >> 'date'

<br>

I'm not very keen on the typing of this next piece of code as it's harder to follow what's going on:

In [11]:
pdf[['date', '1y']] >> TT;
pdf[['date']] >> TT;
pdf['date'] >> TT;

As an example, consider what the type is of `pdf[x]`. You can't tell from the syntax of the snippet and first have to figure the type of `x`.